In [2]:
import pandas as pd
import numpy as np

import pytz
import astral as ast
from astral import LocationInfo
from astral.sun import sun
from datetime import datetime, timedelta, date

In [3]:
# read in with pandas
winter1 = pd.read_csv ('oxc20-21data.txt')
winter2 = pd.read_csv ('oxc21-22data.txt')
winter3 = pd.read_csv ('oxc22-23data.txt')

In [4]:
# rename columns
winter1 = winter1.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter2 = winter2.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter3 = winter3.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})

# datetime format
winter1['datetime'] = pd.to_datetime(winter1['datetime'])
winter2['datetime'] = pd.to_datetime(winter2['datetime'])
winter3['datetime'] = pd.to_datetime(winter3['datetime'])

In [5]:
# mask out days without precipitation
codes = ['GR','GS','IC','PL','PY','RA','SG','SH','SN','SQ','TS','UP']

mask1 = winter1['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask2 = winter2['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask3 = winter3['wxcodes'].apply(lambda x: any(sub in x for sub in codes))

precip_days1 = winter1[mask1]
precip_days2 = winter2[mask2]
precip_days3 = winter3[mask3]

In [6]:
# replace Trace with 0.0001
precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
precip_days3['precip(mm)'] = precip_days3['precip(mm)'].replace('T', 0.0001)

C:\Users\alyha\AppData\Local\Temp\ipykernel_9552\3060002225.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_9552\3060002225.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_9552\3060002225.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a 

In [7]:
# make sure numeric form
precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
precip_days3['precip(mm)'] = pd.to_numeric(precip_days3['precip(mm)'])

C:\Users\alyha\AppData\Local\Temp\ipykernel_9552\2630444437.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_9552\2630444437.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_9552\2630444437.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a

In [8]:
# eliminate special weather 
precip_days1 = precip_days1[precip_days1['datetime'].dt.minute == 51]
precip_days2 = precip_days2[precip_days2['datetime'].dt.minute == 51]
precip_days3 = precip_days3[precip_days3['datetime'].dt.minute == 51]

In [9]:
precip_days1.dtypes

station               object
datetime      datetime64[ns]
precip(mm)           float64
wxcodes               object
dtype: object

In [10]:
precip_days1['date'] = precip_days1['datetime'].dt.date

In [11]:
oxc = LocationInfo(
    'Oxford', 'Connecticut', 
    latitude=41.4338,
    longitude=- 73.1168,
    timezone='America/Connecticut'
)

oxc

LocationInfo(name='Oxford', region='Connecticut', timezone='America/Connecticut', latitude=41.4338, longitude=-73.1168)

In [12]:
# create a date range (winter 1)
date_range1 = pd.date_range(start='2020-12-01', end='2021-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times1 = []
sunset_times1 = []

# Calculate sunrise and sunset for each date
for date in date_range1:
    s = sun(oxc.observer, date=date)
    
    sunrise_times1.append(s['sunrise'])
    sunset_times1.append(s['sunset'])

# df with results
sun_times1 = pd.DataFrame({
    'datetime': date_range1.date,
    'sunrise': sunrise_times1,
    'sunset': sunset_times1
})

sun_times1['sunrise'] = pd.to_datetime(sun_times1['sunrise'])
sun_times1['sunset'] = pd.to_datetime(sun_times1['sunset'])

In [13]:
# create a date range (winter 2)
date_range2 = pd.date_range(start='2021-12-01', end='2022-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times2 = []
sunset_times2 = []

# Calculate sunrise and sunset for each date
for date in date_range2:
    s = sun(oxc.observer, date=date)
    
    sunrise_times2.append(s['sunrise'])
    sunset_times2.append(s['sunset'])

# df with results
sun_times2 = pd.DataFrame({
    'datetime': date_range2.date,
    'sunrise': sunrise_times2,
    'sunset': sunset_times2
})

sun_times2['sunrise'] = pd.to_datetime(sun_times2['sunrise'])
sun_times2['sunset'] = pd.to_datetime(sun_times2['sunset'])

In [14]:
# create a date range (winter 3)
date_range3 = pd.date_range(start='2022-12-01', end='2023-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times3 = []
sunset_times3 = []

# Calculate sunrise and sunset for each date
for date in date_range3:
    s = sun(oxc.observer, date=date)
    
    sunrise_times3.append(s['sunrise'])
    sunset_times3.append(s['sunset'])

# df with results
sun_times3 = pd.DataFrame({
    'datetime': date_range3.date,
    'sunrise': sunrise_times3,
    'sunset': sunset_times3
})

sun_times3['sunrise'] = pd.to_datetime(sun_times3['sunrise'])
sun_times3['sunset'] = pd.to_datetime(sun_times3['sunset'])

In [15]:
# add sunrise and sunset times (winter1)
sun_times1['sunrise'] = sun_times1['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times1['sunset'] = sun_times1['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times1['date'] = sun_times1['datetime']
sun_times1 = sun_times1.drop(columns=['datetime'])

print(sun_times1)

                sunrise               sunset        date
0   2020-12-01 07:00:01  2020-12-01 16:23:13  2020-12-01
1   2020-12-02 07:01:02  2020-12-02 16:22:59  2020-12-02
2   2020-12-03 07:02:03  2020-12-03 16:22:48  2020-12-03
3   2020-12-04 07:03:02  2020-12-04 16:22:38  2020-12-04
4   2020-12-05 07:03:59  2020-12-05 16:22:31  2020-12-05
..                  ...                  ...         ...
85  2021-02-24 06:34:30  2021-02-24 17:37:18  2021-02-24
86  2021-02-25 06:32:59  2021-02-25 17:38:30  2021-02-25
87  2021-02-26 06:31:28  2021-02-26 17:39:41  2021-02-26
88  2021-02-27 06:29:56  2021-02-27 17:40:52  2021-02-27
89  2021-02-28 06:28:23  2021-02-28 17:42:02  2021-02-28

[90 rows x 3 columns]


In [16]:
# add sunrise and sunset times (winter2)

sun_times2['sunrise'] = sun_times2['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times2['sunset'] = sun_times2['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times2['date'] = sun_times2['datetime']
sun_times2 = sun_times2.drop(columns=['datetime'])

print(sun_times2)

                sunrise               sunset        date
0   2021-12-01 06:59:46  2021-12-01 16:23:17  2021-12-01
1   2021-12-02 07:00:48  2021-12-02 16:23:02  2021-12-02
2   2021-12-03 07:01:48  2021-12-03 16:22:50  2021-12-03
3   2021-12-04 07:02:47  2021-12-04 16:22:40  2021-12-04
4   2021-12-05 07:03:46  2021-12-05 16:22:32  2021-12-05
..                  ...                  ...         ...
85  2022-02-24 06:34:51  2022-02-24 17:37:01  2022-02-24
86  2022-02-25 06:33:21  2022-02-25 17:38:12  2022-02-25
87  2022-02-26 06:31:50  2022-02-26 17:39:24  2022-02-26
88  2022-02-27 06:30:18  2022-02-27 17:40:35  2022-02-27
89  2022-02-28 06:28:45  2022-02-28 17:41:45  2022-02-28

[90 rows x 3 columns]


In [17]:
# add sunrise and sunset times (winter3)

sun_times3['sunrise'] = sun_times3['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times3['sunset'] = sun_times3['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times3['date'] = sun_times3['datetime']
sun_times3 = sun_times3.drop(columns=['datetime'])

print(sun_times3)

                sunrise               sunset        date
0   2022-12-01 06:59:31  2022-12-01 16:23:20  2022-12-01
1   2022-12-02 07:00:33  2022-12-02 16:23:05  2022-12-02
2   2022-12-03 07:01:34  2022-12-03 16:22:52  2022-12-03
3   2022-12-04 07:02:33  2022-12-04 16:22:42  2022-12-04
4   2022-12-05 07:03:32  2022-12-05 16:22:34  2022-12-05
..                  ...                  ...         ...
85  2023-02-24 06:35:13  2023-02-24 17:36:43  2023-02-24
86  2023-02-25 06:33:43  2023-02-25 17:37:55  2023-02-25
87  2023-02-26 06:32:12  2023-02-26 17:39:06  2023-02-26
88  2023-02-27 06:30:40  2023-02-27 17:40:17  2023-02-27
89  2023-02-28 06:29:08  2023-02-28 17:41:28  2023-02-28

[90 rows x 3 columns]


In [18]:
# add 'date' column to all dataframes
precip_days1['date'] = precip_days1['datetime'].dt.date
precip_days2['date'] = precip_days2['datetime'].dt.date
precip_days3['date'] = precip_days3['datetime'].dt.date

In [19]:
# merge precip days and sunrise/sunset times on date 
merged1 = pd.merge(precip_days1, sun_times1, on='date')

merged1

station            datetime  precip(mm) wxcodes        date  \
0       OXC 2020-12-01 00:51:00      2.0300  -RA BR  2020-12-01   
1       OXC 2020-12-01 01:51:00      0.0001  -RA BR  2020-12-01   
2       OXC 2020-12-01 02:51:00      0.7600  -RA BR  2020-12-01   
3       OXC 2020-12-04 14:51:00      0.0001     -RA  2020-12-04   
4       OXC 2020-12-04 21:51:00      0.0001     -RA  2020-12-04   
..      ...                 ...         ...     ...         ...   
309     OXC 2021-02-27 15:51:00      1.7800  -RA BR  2021-02-27   
310     OXC 2021-02-27 16:51:00      2.0300  -RA BR  2021-02-27   
311     OXC 2021-02-27 17:51:00      1.7800  -RA BR  2021-02-27   
312     OXC 2021-02-27 18:51:00      0.0001  -RA BR  2021-02-27   
313     OXC 2021-02-27 19:51:00      0.0001  -RA FG  2021-02-27   

                 sunrise               sunset  
0    2020-12-01 07:00:01  2020-12-01 16:23:13  
1    2020-12-01 07:00:01  2020-12-01 16:23:13  
2    2020-12-01 07:00:01  2020-12-01 16:23:13  
3    2020-12-04 07:03:02  2020-12-04 16:22:38  
4    2020-12-04 07:03:02  2020-12-04 16:22:38  
..                   ...                  ...  
309  2021-02-27 06:29:56  2021-02-27 17:40:52  
310  2021-02-27 06:29:56  2021-02-27 17:40:52  
311  2021-02-27 06:29:56  2021-02-27 17:40:52  
312  2021-02-27 06:29:56  2021-02-27 17:40:52  
313  2021-02-27 06:29:56  2021-02-27 17:40:52  

[314 rows x 7 columns]

In [20]:
# merge precip days and sunrise/sunset times on date 
merged2 = pd.merge(precip_days2, sun_times2, on='date')

merged2

station            datetime  precip(mm) wxcodes        date  \
0       OXC 2021-12-02 06:51:00      0.0001     -RA  2021-12-02   
1       OXC 2021-12-02 07:51:00      0.0001     -RA  2021-12-02   
2       OXC 2021-12-02 08:51:00      0.5100  -RA BR  2021-12-02   
3       OXC 2021-12-02 09:51:00      0.0001  -RA BR  2021-12-02   
4       OXC 2021-12-02 19:51:00      0.0001     -RA  2021-12-02   
..      ...                 ...         ...     ...         ...   
313     OXC 2022-02-25 14:51:00      0.0001  -SN BR  2022-02-25   
314     OXC 2022-02-25 15:51:00      0.0001     -SN  2022-02-25   
315     OXC 2022-02-25 18:51:00      4.5700  -SN BR  2022-02-25   
316     OXC 2022-02-25 22:51:00      0.0001     -RA  2022-02-25   
317     OXC 2022-02-25 23:51:00      0.0001     -SN  2022-02-25   

                 sunrise               sunset  
0    2021-12-02 07:00:48  2021-12-02 16:23:02  
1    2021-12-02 07:00:48  2021-12-02 16:23:02  
2    2021-12-02 07:00:48  2021-12-02 16:23:02  
3    2021-12-02 07:00:48  2021-12-02 16:23:02  
4    2021-12-02 07:00:48  2021-12-02 16:23:02  
..                   ...                  ...  
313  2022-02-25 06:33:21  2022-02-25 17:38:12  
314  2022-02-25 06:33:21  2022-02-25 17:38:12  
315  2022-02-25 06:33:21  2022-02-25 17:38:12  
316  2022-02-25 06:33:21  2022-02-25 17:38:12  
317  2022-02-25 06:33:21  2022-02-25 17:38:12  

[318 rows x 7 columns]

In [21]:
# merge precip days and sunrise/sunset times on date 
merged3 = pd.merge(precip_days3, sun_times3, on='date')

merged3

station            datetime  precip(mm) wxcodes        date  \
0       OXC 2022-12-01 11:51:00      0.0001     -RA  2022-12-01   
1       OXC 2022-12-03 10:51:00      0.0001  -RA BR  2022-12-03   
2       OXC 2022-12-03 11:51:00      0.2500  -RA BR  2022-12-03   
3       OXC 2022-12-03 12:51:00      0.0001  -RA BR  2022-12-03   
4       OXC 2022-12-03 13:51:00      0.5100  -RA BR  2022-12-03   
..      ...                 ...         ...     ...         ...   
267     OXC 2023-02-25 15:51:00      0.0001     -SN  2023-02-25   
268     OXC 2023-02-25 16:51:00      0.0001     -SN  2023-02-25   
269     OXC 2023-02-25 17:51:00      0.0001     -SN  2023-02-25   
270     OXC 2023-02-25 18:51:00      0.2500     -SN  2023-02-25   
271     OXC 2023-02-25 23:51:00      0.0001     -SN  2023-02-25   

                 sunrise               sunset  
0    2022-12-01 06:59:31  2022-12-01 16:23:20  
1    2022-12-03 07:01:34  2022-12-03 16:22:52  
2    2022-12-03 07:01:34  2022-12-03 16:22:52  
3    2022-12-03 07:01:34  2022-12-03 16:22:52  
4    2022-12-03 07:01:34  2022-12-03 16:22:52  
..                   ...                  ...  
267  2023-02-25 06:33:43  2023-02-25 17:37:55  
268  2023-02-25 06:33:43  2023-02-25 17:37:55  
269  2023-02-25 06:33:43  2023-02-25 17:37:55  
270  2023-02-25 06:33:43  2023-02-25 17:37:55  
271  2023-02-25 06:33:43  2023-02-25 17:37:55  

[272 rows x 7 columns]

In [22]:
# filter out only times after sunrise & before sunset 

days1 = merged1[(merged1['datetime'] >= merged1['sunrise']) & (merged1['datetime'] <= merged1['sunset'])]
days2 = merged2[(merged2['datetime'] >= merged2['sunrise']) & (merged2['datetime'] <= merged2['sunset'])]
days3 = merged3[(merged3['datetime'] >= merged3['sunrise']) & (merged3['datetime'] <= merged3['sunset'])]


days1

station            datetime  precip(mm) wxcodes        date  \
3       OXC 2020-12-04 14:51:00      0.0001     -RA  2020-12-04   
14      OXC 2020-12-05 07:51:00      0.2500  -RA BR  2020-12-05   
15      OXC 2020-12-05 08:51:00      0.7600  -RA BR  2020-12-05   
16      OXC 2020-12-05 09:51:00      2.5400  -RA BR  2020-12-05   
17      OXC 2020-12-05 10:51:00      4.0600      RA  2020-12-05   
..      ...                 ...         ...     ...         ...   
306     OXC 2021-02-27 12:51:00      0.5100  -RA BR  2021-02-27   
307     OXC 2021-02-27 13:51:00      0.5100   UP BR  2021-02-27   
308     OXC 2021-02-27 14:51:00      1.0200  -RA BR  2021-02-27   
309     OXC 2021-02-27 15:51:00      1.7800  -RA BR  2021-02-27   
310     OXC 2021-02-27 16:51:00      2.0300  -RA BR  2021-02-27   

                 sunrise               sunset  
3    2020-12-04 07:03:02  2020-12-04 16:22:38  
14   2020-12-05 07:03:59  2020-12-05 16:22:31  
15   2020-12-05 07:03:59  2020-12-05 16:22:31  
16   2020-12-05 07:03:59  2020-12-05 16:22:31  
17   2020-12-05 07:03:59  2020-12-05 16:22:31  
..                   ...                  ...  
306  2021-02-27 06:29:56  2021-02-27 17:40:52  
307  2021-02-27 06:29:56  2021-02-27 17:40:52  
308  2021-02-27 06:29:56  2021-02-27 17:40:52  
309  2021-02-27 06:29:56  2021-02-27 17:40:52  
310  2021-02-27 06:29:56  2021-02-27 17:40:52  

[143 rows x 7 columns]

In [23]:
# filter out transition hours
days1['sunrise'] = pd.to_datetime(days1['sunrise'])
days1['sunset'] = pd.to_datetime(days1['sunset'])

days1['sunrise+1'] = days1['sunrise'] + timedelta(hours = 1)
days1['sunset-1'] = days1['sunset'] - timedelta(hours = 1)

days1 = days1[days1['datetime'] <= days1['sunset-1']]
days1 = days1[days1['datetime'] >= days1['sunrise+1']]



days2['sunrise'] = pd.to_datetime(days2['sunrise'])
days2['sunset'] = pd.to_datetime(days2['sunset'])

days2['sunrise+1'] = days2['sunrise'] + timedelta(hours = 1)
days2['sunset-1'] = days2['sunset'] - timedelta(hours = 1)

days2 = days2[days2['datetime'] <= days2['sunset-1']]
days2 = days2[days2['datetime'] >= days2['sunrise+1']]



days3['sunrise'] = pd.to_datetime(days3['sunrise'])
days3['sunset'] = pd.to_datetime(days3['sunset'])

days3['sunrise+1'] = days3['sunrise'] + timedelta(hours = 1)
days3['sunset-1'] = days3['sunset'] - timedelta(hours = 1)

days3 = days3[days3['datetime'] <= days3['sunset-1']]
days3 = days3[days3['datetime'] >= days3['sunrise+1']]

C:\Users\alyha\AppData\Local\Temp\ipykernel_9552\2882998222.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunrise'] = pd.to_datetime(days1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_9552\2882998222.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunset'] = pd.to_datetime(days1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_9552\2882998222.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,c

In [24]:
days1

station            datetime  precip(mm) wxcodes        date  \
3       OXC 2020-12-04 14:51:00      0.0001     -RA  2020-12-04   
15      OXC 2020-12-05 08:51:00      0.7600  -RA BR  2020-12-05   
16      OXC 2020-12-05 09:51:00      2.5400  -RA BR  2020-12-05   
17      OXC 2020-12-05 10:51:00      4.0600      RA  2020-12-05   
18      OXC 2020-12-05 11:51:00      4.5700      RA  2020-12-05   
..      ...                 ...         ...     ...         ...   
305     OXC 2021-02-27 11:51:00      0.0001  -RA BR  2021-02-27   
306     OXC 2021-02-27 12:51:00      0.5100  -RA BR  2021-02-27   
307     OXC 2021-02-27 13:51:00      0.5100   UP BR  2021-02-27   
308     OXC 2021-02-27 14:51:00      1.0200  -RA BR  2021-02-27   
309     OXC 2021-02-27 15:51:00      1.7800  -RA BR  2021-02-27   

                sunrise              sunset           sunrise+1  \
3   2020-12-04 07:03:02 2020-12-04 16:22:38 2020-12-04 08:03:02   
15  2020-12-05 07:03:59 2020-12-05 16:22:31 2020-12-05 08:03:59   
16  2020-12-05 07:03:59 2020-12-05 16:22:31 2020-12-05 08:03:59   
17  2020-12-05 07:03:59 2020-12-05 16:22:31 2020-12-05 08:03:59   
18  2020-12-05 07:03:59 2020-12-05 16:22:31 2020-12-05 08:03:59   
..                  ...                 ...                 ...   
305 2021-02-27 06:29:56 2021-02-27 17:40:52 2021-02-27 07:29:56   
306 2021-02-27 06:29:56 2021-02-27 17:40:52 2021-02-27 07:29:56   
307 2021-02-27 06:29:56 2021-02-27 17:40:52 2021-02-27 07:29:56   
308 2021-02-27 06:29:56 2021-02-27 17:40:52 2021-02-27 07:29:56   
309 2021-02-27 06:29:56 2021-02-27 17:40:52 2021-02-27 07:29:56   

               sunset-1  
3   2020-12-04 15:22:38  
15  2020-12-05 15:22:31  
16  2020-12-05 15:22:31  
17  2020-12-05 15:22:31  
18  2020-12-05 15:22:31  
..                  ...  
305 2021-02-27 16:40:52  
306 2021-02-27 16:40:52  
307 2021-02-27 16:40:52  
308 2021-02-27 16:40:52  
309 2021-02-27 16:40:52  

[119 rows x 9 columns]

In [25]:
# find total precip over each year
day_total1 = days1['precip(mm)'].sum()
day_total2 = days2['precip(mm)'].sum()
day_total3 = days3['precip(mm)'].sum()

# total number of obs each year
day_rows1 = days1.shape[0]
day_rows2 = days2.shape[0]
day_rows3 = days3.shape[0]

In [26]:
day_precip_per_hour1 = day_total1/ day_rows1
day_precip_per_hour2 = day_total2/ day_rows2
day_precip_per_hour3 = day_total3/ day_rows3

In [27]:
print('2020-2021:', day_precip_per_hour1)
print('2021-2022:', day_precip_per_hour2)
print('2022-2023:', day_precip_per_hour3)

2020-2021: 0.7446815126050421
2021-2022: 0.5390603305785124
2022-2023: 0.6277528571428572


In [28]:
# filter out only times after sunset & before NEXT sunrise 

nights1 = merged1[(merged1['datetime'] < merged1['sunrise']) | (merged1['datetime'] > merged1['sunset'])]
nights2 = merged2[(merged2['datetime'] < merged2['sunrise']) | (merged2['datetime'] > merged2['sunset'])]
nights3 = merged3[(merged3['datetime'] < merged3['sunrise']) | (merged3['datetime'] > merged3['sunset'])]

nights1

station            datetime  precip(mm) wxcodes        date  \
0       OXC 2020-12-01 00:51:00      2.0300  -RA BR  2020-12-01   
1       OXC 2020-12-01 01:51:00      0.0001  -RA BR  2020-12-01   
2       OXC 2020-12-01 02:51:00      0.7600  -RA BR  2020-12-01   
4       OXC 2020-12-04 21:51:00      0.0001     -RA  2020-12-04   
5       OXC 2020-12-04 22:51:00      0.0001     -RA  2020-12-04   
..      ...                 ...         ...     ...         ...   
300     OXC 2021-02-25 03:51:00      0.0001     -RA  2021-02-25   
301     OXC 2021-02-25 04:51:00      0.0001     -RA  2021-02-25   
311     OXC 2021-02-27 17:51:00      1.7800  -RA BR  2021-02-27   
312     OXC 2021-02-27 18:51:00      0.0001  -RA BR  2021-02-27   
313     OXC 2021-02-27 19:51:00      0.0001  -RA FG  2021-02-27   

                 sunrise               sunset  
0    2020-12-01 07:00:01  2020-12-01 16:23:13  
1    2020-12-01 07:00:01  2020-12-01 16:23:13  
2    2020-12-01 07:00:01  2020-12-01 16:23:13  
4    2020-12-04 07:03:02  2020-12-04 16:22:38  
5    2020-12-04 07:03:02  2020-12-04 16:22:38  
..                   ...                  ...  
300  2021-02-25 06:32:59  2021-02-25 17:38:30  
301  2021-02-25 06:32:59  2021-02-25 17:38:30  
311  2021-02-27 06:29:56  2021-02-27 17:40:52  
312  2021-02-27 06:29:56  2021-02-27 17:40:52  
313  2021-02-27 06:29:56  2021-02-27 17:40:52  

[171 rows x 7 columns]

In [29]:
# filter out transition hours
nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
nights1['sunset'] = pd.to_datetime(nights1['sunset'])

nights1['sunrise-1'] = nights1['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights1['sunset+1'] = nights1['sunset'] + timedelta(hours = 1)

nights1 = nights1[nights1['datetime'] >= nights1['sunset+1']]
nights1 = nights1[nights1['datetime'] <= nights1['sunrise-1']]



nights2['sunrise'] = pd.to_datetime(nights2['sunrise'])
nights2['sunset'] = pd.to_datetime(nights2['sunset'])

nights2['sunrise-1'] = nights2['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights2['sunset+1'] = nights2['sunset'] + timedelta(hours = 1)

nights2 = nights2[nights2['datetime'] >= nights2['sunset+1']]
nights2 = nights2[nights2['datetime'] <= nights2['sunrise-1']]



nights3['sunrise'] = pd.to_datetime(nights3['sunrise'])
nights3['sunset'] = pd.to_datetime(nights3['sunset'])

nights3['sunrise-1'] = nights3['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights3['sunset+1'] = nights3['sunset'] + timedelta(hours = 1)

nights3 = nights3[nights3['datetime'] >= nights3['sunset+1']]
nights3 = nights3[nights3['datetime'] <= nights3['sunrise-1']]


C:\Users\alyha\AppData\Local\Temp\ipykernel_9552\36922422.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_9552\36922422.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunset'] = pd.to_datetime(nights1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_9552\36922422.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer

In [30]:
# find total precip over each year
night_total1 = nights1['precip(mm)'].sum()
night_total2 = nights2['precip(mm)'].sum()
night_total3 = nights3['precip(mm)'].sum()

# total number of obs each year
night_rows1 = nights1.shape[0]
night_rows2 = nights2.shape[0]
night_rows3 = nights3.shape[0]

In [31]:
night_precip_per_hour1 = night_total1/ night_rows1
night_precip_per_hour2 = night_total2/ night_rows2
night_precip_per_hour3 = night_total3/ night_rows3

In [32]:
print('2020-2021:', night_precip_per_hour1)
print('2021-2022:', night_precip_per_hour2)
print('2022-2023:', night_precip_per_hour3)

2020-2021: 0.35651518987341774
2021-2022: 0.5490074626865673
2022-2023: 0.49061011235955054
